# 0) User Input

In [ ]:
Regressor_File = 'data-PromLib_EcolPtai/00000000_PromLib_EcolPtai_Ecol-Promoter-Activity_SVR-Regressor.pkl'#'RegressorTests/SVR-Regressor-Standard.pkl'
Scaler_File = 'data-PromLib_EcolPtai/00000000_PromLib_EcolPtai_Ecol-Promoter-Activity_SVR-Params.pkl'#'RegressorTests/SVR-Scaler.pkl'

output_filepath = "modelfile"
output_filename = "gene-svr"

# 1) Generate Model file

In [ ]:
import joblib
import utils
import timeit


In [ ]:
# Unpickle regressor and scaler
myRegr = joblib.load(Regressor_File)
myScaler = joblib.load(Scaler_File)

# Create dictionary containing the output scaler
scaler = {'output': myScaler['Ecol Promoter Activity_Scaler']} #Ecol Promoter Activity_Scaler, Promoter Activity_ML_Scaler

# Create model file
utils.save_model_to_json(output_filepath, output_filename + '.json', myRegr, scaler)

In [ ]:
# Save number of nukleotides the regressor uses as input, this is required to specifiy the number 
# of optimization variables
nNukleotides = myRegr.support_vectors_.shape[1] - 1
print('Input vector: {}, nucleotides: {}'.format(nNukleotides, nNukleotides/4))

# 2) Optimize Model

## Imports

In [ ]:
from _pymaingo import *
from pymelon import *

## Define Problem

In [ ]:
# To specify the optimization problem to be solved by MAiNGO
# we need to implement a specialization of the abstract MAiNGOmodel class.
# In particular, we need to implement the get_variables and evaluate functions.
# In total, this model needs to implement an (MI)NLP optimization problem of the format
# min_{x \in X} f(x)
#      s.t.     h(x)  = 0
#               g(x) <= 0,
# where X (the host set) is a compact hyperrectangle in R^n (or, optionally,
# some entries of x can be restricted to integer values).
class Model(MAiNGOmodel):

    #####################################
    # This function defines the optimization variables in the problem. In the above (MI)NLP, this corresponds to the x \in X part.
    # A variable typically needs bounds to define the host set X and can optionally have a variable type (VT_CONTINUOUS, VT_INTEGER, VT_BINARY)
    # and a name (and a branching priority, which is an algorithmic parameter and usually not relevant).
    # Note that the name is used *only* in the screen or log output of MAiNGO. In particular, it can not be used for modelling...
    def get_variables(self):
        # Add a binary variable for each of the nuleotides
        return [OptimizationVariable(VT_BINARY, "x_" + str(i)) for i in range(nNukleotides)]

    #####################################
    # This function essentially needs to implement the functions f(x), h(x), and g(x) in the aove (MI)NLP.
    # Unfortunaley, right now we cannot use the variable objects defined in the get_variables function above
    # directly for modeling (nor other variable types defined elsewhere) for technical reasons.
    # Instead, the "vars" list that MAiNGO hands to this evaluate function contains the same number of elements as the
    # list that we returned in the get_variables function, and it is only through their position in this list that we can
    # map the variables used herein to the ones we defined in get_variables.
    def evaluate(self,vars):
        
        # Load support vector regressor from file
        svr = SupportVectorRegression(output_filepath, output_filename)

        # Calculate the vc share and append it to the 
        gc_share =0
        for i in range(0,nNukleotides,4):
            gc_share += vars[i+1] + vars[i+2]
            
        gc_share /= nNukleotides
        
        vars.append(gc_share)
        
        # Assign the evaluation of the regressor as the objective
        expr = svr.calculate_prediction_reduced_space(vars)
        
        # Add constraint that ensures that at each position only one nukleotide is active
        constraints = []
        for i in range(0,nNukleotides,4):
            constraints.append(vars[i+1] + vars[i+2] + vars[i+3] + vars[i] - 1)
        

        # Create a evaluation container with the defined objective and constraint functions
        result = EvaluationContainer()
        result.eq = constraints
#         result.objective = -expr;    # Maingo minimizes by default, to maximize we have to pass the objective as negative
        target = .3
        result.objective = (expr-target)**2;    # Maingo minimizes by default, to maximize we have to pass the objective as negative

        # Optionally, we can define OutputVariables. These are things that we would like to have
        # evaluated at the optimal solution but that do not form part of the (MI)NLP itself.
        # 
        result.output = [ OutputVariable('Expression: ', expr)]
        return result

## Perform Optimization

In [ ]:
# To actually solve the problem, we instantiate a model, hand it to a MAiNGO instance.
myModel = Model()
myMAiNGO = MAiNGO(myModel)

In [ ]:

start_time = timeit.default_timer()
# There are lots of settings regarding algorithmic details, subsolvers, and output.
# E.g., to disable screen output:
myMAiNGO.set_option("outstreamVerbosity", 1)
myMAiNGO.set_option("maxTime", 360)
myMAiNGO.set_option("PRE_maxLocalSearches", 50)

# Actually solve the problem
myMAiNGO.solve()

# Query results
optimalObjective = myMAiNGO.get_objective_value()
print("\nOptimal objective value: {}".format(-optimalObjective))
optimalSolutionPoint = myMAiNGO.get_solution_point()
run_time = timeit.default_timer() - start_time

print('Optimization run time: {:.0f} sec'.format(run_time))
print("Optimal solution point: {}\n".format(optimalSolutionPoint))


In [ ]:
import numpy as np
gc_share =0
for i in range(0,nNukleotides,4):
    gc_share += optimalSolutionPoint[i+1] + optimalSolutionPoint[i+2]

gc_share /= nNukleotides

newstr = np.append(optimalSolutionPoint[:-2],gc_share)
OptExpr_stand = myRegr.predict([newstr])
Scaler = myScaler['Ecol Promoter Activity_Scaler']
myResult = Scaler.inverse_transform(OptExpr_stand)


In [ ]:
OptExpr_stand = myRegr.predict([optimalSolutionPoint])
Scaler = myScaler['Ecol Promoter Activity_Scaler']
myResult = Scaler.inverse_transform(OptExpr_stand)

print('result: {}'.format(myResult))